# Numba Introduction

## What is Numba?
Numba is a just-in-time, type-specializing, function compiler for accelerating numerically-focused Python. That's a long list, so let's break down those terms:

* __function compiler__: Numba compiles Python functions, not entire applications, and not parts of functions. Numba does not replace your Python interpreter, but is just another Python module that can turn a function into a (usually) faster function.
* __type-specializing__: Numba speeds up your function by generating a specialized implementation for the specific data types you are using. Python functions are designed to operate on generic data types, which makes them very flexible, but also very slow. In practice, you only will call a function with a small number of argument types, so Numba will generate a fast implementation for each set of types.
* __just-in-time__: Numba translates functions when they are first called. This ensures the compiler knows what argument types you will be using. This also allows Numba to be used interactively in a Jupyter notebook just as easily as a traditional application.
* __numerically-focused__: Currently, Numba is focused on numerical data types, like int, float, and complex. There is very limited string processing support, and many string use cases are not going to work well on the GPU. To get best results with Numba, you will likely be using NumPy arrays.

## Setup

In [2]:
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Problem I - First Numba Function
To start our exploration of Numba's features, let's write a python function to add two numbers. We'll creatively name it add:

In [3]:
def add(a,b):
    return a + b

Just by looking at this function a clear issue comes to mind, lets demonstrate:

In [4]:
import numpy as np

print("Int: ", add(1, 2)) ## Add can sum up integers
print("Float: ", add(1.5, 2.3)) ## Add can sum up floats
print("Numpy: ", add(np.array([1,2,3]), np.array([3,4,5]))) ## Add can sum up numpy arrays

print("Lists: ", add([1,2], [2,3])) ## Add can concate two python list


Int:  3
Float:  3.8
Numpy:  [4 6 8]
Lists:  [1, 2, 2, 3]


That means that python spends much time on being type agnostic, lets use numba to perform jit on add function

In [5]:
from numba import jit

numba_add = jit(add) # wrap function with jit function to generate just in time function

@jit # one can also use the decorator which essentially does the same thing
def add_test(a, b):
    return a + b

### Exercise 
Numba stores the old function in a class member called 'py_func', lets compare the results to see it fits

### Performance Test
%timeit is running our function many times, and then reporting the average time it takes to run. This is generally a better approach than timing a single function execution, because it accounts for random events that may cause any given run to perform poorly.

Lets time the original function:

In [6]:
%timeit add(1,2)

10000000 loops, best of 5: 87.3 ns per loop


Time the jit function:

In [7]:
%timeit numba_add(1,2)

The slowest run took 818579.81 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 297 ns per loop


Hold on - our new pre-compiled function is running even slower than the original python version! What's going on here?
Numba isn't going to speed up everything. Generally, Numba will help you most in circumstances where python's line-by-line interperability and lack of type casting is slowing it down.

Lets do more complicated function to demonstrate the difference:

In [8]:
import math

def hypotenuse(x, y):
  x = abs(x);
  y = abs(y);
  t = min(x, y);
  x = max(x, y);
  t = t / x;
  return x * math.sqrt(1+t*t)

### Exercise
Convert the function hypotenuse to numba function and compare timing with original function

In [9]:
numba_hypotenuse = jit(hypotenuse)
%timeit hypotenuse(3.4,4.3)

The slowest run took 6.39 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 675 ns per loop


In [10]:
%timeit numba_hypotenuse(3.4,4.3)

The slowest run took 605072.99 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 206 ns per loop


Numba functions can call other functions, provided they are also Numba functions.

**No Python mode vs Object mode** - 
A common pattern is to decorate functions with @jit as this is the most flexible decorator offered by Numba. @jit essentially encompasses two modes of compilation, first it will try and compile the decorated function in no Python mode, if this fails it will try again to compile the function using object mode. Whilst the use of looplifting in object mode can enable some performance increase, getting functions to compile under no python mode is really the key to good performance. To make it such that only no python mode is used and if compilation fails an exception is raised the decorators @njit and @jit(nopython=True) can be used (the first is an alias of the second for convenience).

> **Note**: The behaviour of the nopython compilation mode is to essentially compile the decorated function so that it will run entirely without the involvement of the Python interpreter. This is the recommended and best-practice way to use the Numba jit decorator as it leads to the best performance.

Numba, compiles each function based on the types of its arguments, and infers the type of the result. You can see this if you run the inspect_types function on a numba function:

In [11]:
numba_add.inspect_types()

add (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-3-494b6d29bcde>
# --- LINE 1 --- 

def add(a,b):

    # --- LINE 2 --- 
    # label 0
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $6binary_add.2 = a + b  :: int64
    #   del b
    #   del a
    #   $8return_value.3 = cast(value=$6binary_add.2)  :: int64
    #   del $6binary_add.2
    #   return $8return_value.3

    return a + b




In [12]:
numba_hypotenuse.inspect_types()

hypotenuse (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-8-9edeb24715a4>
# --- LINE 3 --- 

def hypotenuse(x, y):

  # --- LINE 4 --- 
  # label 0
  #   x = arg(0, name=x)  :: float64
  #   y = arg(1, name=y)  :: float64
  #   $2load_global.0 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
  #   x.1 = call $2load_global.0(x, func=$2load_global.0, args=[Var(x, <ipython-input-8-9edeb24715a4>:4)], kws=(), vararg=None, varkwarg=None, target=None)  :: (float64,) -> float64
  #   del x
  #   del $2load_global.0

  x = abs(x);

  # --- LINE 5 --- 
  #   $10load_global.3 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
  #   y.1 = call $10load_global.3(y, func=$10load_global.3, args=[Var(y, <ipython-input-8-9edeb24715a4>:4)], kws=(), vararg=None, varkwarg=None, target=None)  :: (float64,) -> float64
  #   del y
  #   del $10load_global.3

  y = abs(y);

  # --- 

So far we have been using what Numba refers to as "lazy" (or "call-time") decoration. Basically, we've been letting Numba do the work of figuring out how we're using the function and inferring the types for us. Alternatively, if we know how we are going to use a given function, we can use "eager" (or "compile-time") decoration. To do this, we make use of the vectorize decorator. For example, if we want to make an integer-only version of our addition function, we could write:

In [25]:
from numba import vectorize

@vectorize(['int64(int64, int64)'], target='cpu')
def add_ufunc_cpu(x, y):
  return x + y

In [14]:
#numba_add_ufunc_cpu = jit(add_ufunc_cpu)
#numba_add_ufunc_cpu.inspect_types()

You'll notice a couple of new things here. In the first set of brackets, we have specified both the argument types of the function (those are inside the parentheses), as well as the return type of the function. This is just making explicit what Numba was previously inferring on our behalf. In second set of brackets you'll see that we have specified a 'target' architechture for the function. The default is cpu, which means that Numba is optimizing the function to your specific machine. Other options include parallel, which allows you to take advantage of multicore processors, and cuda

> **Note**: the 'u' prefix in add_ufunc numpy means universal functions, a universal function (or ufunc for short) is a function that operates on ndarrays in an element-by-element fashion, supporting array broadcasting, type casting, and several other standard features. That is, a ufunc is a “vectorized” wrapper for a function that takes a fixed number of specific inputs and produces a fixed number of specific outputs.

### Exercise
Run our add_ufunc with cpu and parallel mode and time the difference

In [26]:
%timeit add_ufunc_cpu(345, 543)

The slowest run took 86.24 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 934 ns per loop


In [23]:
@vectorize(['int64(int64, int64)'], target='parallel')
def add_ufunc_par(x, y):
  return x + y

%timeit add_ufunc_par(345, 543)

The slowest run took 25.42 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 2.92 µs per loop


## GPU interaction with Numba

As mentioned before numba can interact with Nvidia GPU (cuda) and even write fully functional kernels in python API only.

### Problem II - Vector addition running on GPU
Lets run the previous ufunction that we ran before with 'cuda' as a target:

In [24]:
@vectorize(['int64(int64, int64)'], target='cuda')
def add_ufunc_cuda(x, y):
    return x + y

x = np.arange(10)
y = 2 * x

add_ufunc_cuda(x, y)

/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27])


What actually just happened?

* Numba compiled a CUDA kernel to execute the ufunc operation in parallel over all the input elements.
* It allocated GPU memory for the inputs and the output.
* It also copied the input data to the GPU.
* Numba executed the CUDA kernel with the correct kernel dimensions given the input sizes.
* Copied the result back from the GPU to the CPU.
* Returned the result as a NumPy array on the host.

### Exercise
Compare the cuda based function to the CPU based function, measure the execution time difference

In [27]:
%timeit add_ufunc_cpu(x, y)

The slowest run took 59.96 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 538 ns per loop


In [26]:
%timeit add_ufunc_cuda(x, y)

1000 loops, best of 5: 1.02 ms per loop


**Why is the GPU function slower?** 

* Our inputs are too small: the GPU achieves performance through parallelism, operating on thousands of values at once. Our test inputs have only 10 integers. We need a much larger array to even keep the GPU busy.
* Our calculation is too simple: Sending a calculation to the GPU involves quite a bit of overhead compared to calling a function on the CPU. If our calculation does not involve enough math operations ("arithmetic intensity"), then the GPU will spend most of its time waiting for data to move around.
* We copy the data to and from the GPU: While including the copy time can be realistic for a single function, often we want to run several GPU operations in sequence. In those cases, it makes sense to send data to the GPU and keep it there until all of our processing is complete.
* Our data types are larger than necessary: Our example uses int64 when we probably don't need it. Scalar code using data types that are 32 and 64-bit run basically the same speed on the CPU, but 64-bit data types have a significant performance cost on the GPU. Basic arithmetic on 64-bit floats can be anywhere from 2x (Pascal-architecture Tesla) to 24x (Maxwell-architecture GeForce) slower than 32-bit floats. NumPy defaults to 64-bit data types when creating arrays, so it is important to set the dtype attribute or use the ndarray.astype() method to pick 32-bit types when you need them.

As we saw in the last problem, Numba can automatically handle transferring data to and from the GPU for us. However, that's not always what we want. Sometimes we will want to perform several functions in a row on the GPU without transferring the data back to the CPU in between.

Lets take the previous function and change the data type to float32, so it will execute efficiently on our GPU:

In [35]:
@vectorize(['float32(float32, float32)'], target='cuda') # add code here
def add_ufunc_cuda_flt32(x, y):
    return x + y

x = np.arange(10).astype(np.float32)
y = 2 * x

%timeit add_ufunc_cuda_flt32(x, y)

/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


The slowest run took 114.74 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 984 µs per loop


As we saw in the last problem, copying the data to and from the GPU for every function is not necessarily the most efficient way to use the GPU. To address this, Numba provides the to_device function in the cuda module to allocate and copy arrays to the GPU

In [36]:
from numba import cuda

n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x

x_device = cuda.to_device(x)
y_device = cuda.to_device(y)

print(x_device)
print(x_device.shape)
print(x_device.dtype)

(100000,)
float32


x_device and y_device are now Numba "device arrays" that are in many ways equivalent to Numpy ndarrays except that they live in the GPU's global memory, rather than on the CPU. These device arrays can be passed to Numba cuda functions just the way Numpy arrays can, but without the memory copying overhead.

### Exercise
Try running the cuda kernel using the host memory and device memory, time the difference

In [37]:
%timeit add_ufunc_cuda_flt32(x, y)

/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 98 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


1000 loops, best of 5: 1.53 ms per loop


In [38]:
%timeit add_ufunc_cuda_flt32(x_device, y_device)

1000 loops, best of 5: 998 µs per loop


You should see a big performance improvement already, but we are still allocating a device array for the output of the ufunc and copying it back to the host. We can create an output buffer on the GPU with the numba.cuda.device_array() function:

In [39]:
out_device = cuda.device_array(shape=(n,), dtype=np.float32)  # does not initialize the contents, much like np.empty()

And then we can use a special out keyword argument to the ufunc to specify the output buffer:

In [40]:
%timeit add_ufunc_cuda_flt32(x_device, y_device, out=out_device)

1000 loops, best of 5: 851 µs per loop


That way we can sepperate the GPU operation from the memory operations and manage it more efficiently

### Exercise
Remake a new version of the addition ufunc with 32bit floats that targets the cpu. Compare the resulting time to execute with the gpu version you just timed.

In [29]:
from numba import vectorize

@vectorize(['float32(float32, float32)'], target='cpu')
def add_ufunc_cpu_flt32(x, y):
  return x + y

n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x

%timeit add_ufunc_cpu_flt32(x, y)

The slowest run took 5.48 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 69.5 µs per loop
